In [1]:
from datasets import load_dataset
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
from tqdm import tqdm
# from gensim.models import Word2Vec

In [2]:
dataset = load_dataset('ms_marco', 'v1.1')
dataset

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 10047
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 82326
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 9650
    })
})

In [3]:
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

train_df = train_data.to_pandas()
train_df.drop(['query_id', 'query_type', 'wellFormedAnswers'], axis=1, inplace=True)
train_df = train_df[:100] #Using small amount of data
train_df.head()

,answers,passages,query
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body


In [4]:
passage = []
answer = []

for i in range(len(train_df)):
    x = train_df['answers'][i].tolist()
    if len(x)==0:
        x = "-"
    passage.append((train_df['passages'][i]['passage_text']).tolist())
    answer.append(x[0])

train_df['passages'] = passage
train_df['answers'] = answer
train_df

,answers,passages,query
0,Results-Based Accountability is a disciplined ...,"[Since 2007, the RBA's outstanding reputation ...",what is rba
1,Yes,"[In his younger years, Ronald Reagan was a mem...",was ronald reagan a democrat
2,20-25 minutes,"[Sydney, New South Wales, Australia is located...",how long do you need for sydney and surroundin...
3,$11 to $22 per square foot,"[In regards to tile installation costs, consum...",price to install tile in shower
4,Due to symptoms in the body,"[Conclusions: In adult body CT, dose to an org...",why conversion observed in body
...,...,...,...
95,WatchDog.sys is a vital system file used by th...,[WatchDog.sys was originally stored in the sys...,watchdog.sys what is
96,"In computing, .bak is a filename extension com...",[The easiest way to open a BAK file is to doub...,what is a bak file
97,"Public, four-year colleges cost $7,000 for in-...",[A: The degree that you need to be a detective...,How much will it cost to go to college to beco...
98,A document used to change one or more minor pr...,[Trust Restatement Law & Legal Definition. A r...,trust amendment term


In [5]:
train_df = train_df.explode('passages')
train_df = train_df.reset_index(drop = True)
train_df = train_df[['query', 'passages', 'answers']]
train_df

,query,passages,answers
0,what is rba,"Since 2007, the RBA's outstanding reputation h...",Results-Based Accountability is a disciplined ...
1,what is rba,The Reserve Bank of Australia (RBA) came into ...,Results-Based Accountability is a disciplined ...
2,what is rba,RBA Recognized with the 2014 Microsoft US Regi...,Results-Based Accountability is a disciplined ...
3,what is rba,The inner workings of a rebuildable atomizer a...,Results-Based Accountability is a disciplined ...
4,what is rba,Results-Based Accountability® (also known as R...,Results-Based Accountability is a disciplined ...
...,...,...,...
809,what is kuchen,"Some Kuchen's use just a Streusal topping, whi...","Kuchen means cake in German, and refers to a v..."
810,what is kuchen,Kuchen is a tasty dessert with a dough crust a...,"Kuchen means cake in German, and refers to a v..."
811,what is kuchen,Kuchen. Kuchen is a tasty dessert with a dough...,"Kuchen means cake in German, and refers to a v..."
812,what is kuchen,Grandmother's Kuchen. Giora Shimoni. Kuchen me...,"Kuchen means cake in German, and refers to a v..."


In [6]:
i = 65
train_df['query'][i], train_df['answers'][i], train_df['passages'][i]

('what does a metabolic acidosis need to reverse the condition',
 '-',
 'Acidosis is a serious metabolic imbalance in which there is an excess of acidic molecules in the body. This can occur as a result of acid overproduction, impaired acid transport, acid underexcretion, or any combination. With overproduction, the body makes too much acid. This can occur in sepsis, a life-threatening widespread infection in which the body makes too much lactic acid. With underexcretion, the body is unable to rid itself of excess acid. This can occur in renal failure and various lung diseases. In renal failure, the kidneys are unable to cleanse the blood of acid.')

In [7]:
type(train_df['query'][i]), type(train_df['answers'][i]), type(train_df['passages'][i])

(str, str, str)

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, models
import faiss
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim